# **2.6 Elimination = Factorization: A=LU**

- The factors $L$ and $U$ are **triangular matrices**. The factorization that comes from elimination is:

$$
A = LU
$$

- The entries of $L$ are exactly the **multipliers** $l_{ij}$ that multiplied the pivot row $j$ when subtracted from row $i$.  
- Each step from $A$ to $U$ multiplies by a matrix $E_{ij}$ to produce zeros in the $(i, j)$ position. We start with the most frequent case, **no row exchanges**.  

- **Example (3×3 matrix)**: Multiply by $E_{21}$, $E_{31}$, $E_{32}$.  
  The multipliers $l_{ij}$ produce zeros in positions $(2,1)$, $(3,1)$, and $(3,2)$, all **below the diagonal**.  

$$
(E_{32} E_{31} E_{21}) A = U \quad \Rightarrow \quad A = (E_{21}^{-1} E_{31}^{-1} E_{32}^{-1}) U = LU
$$

In [3]:
def lu_decompose(A):
    """
    Compute L and U such that A = L·U
    Doolittle algorithm (L has 1s on the diagonal).
    A must be a square list-of-lists matrix.
    """
    n = len(A)
    if any(len(row) != n for row in A):
        raise ValueError("Matrix must be square.")

    # Initialize L and U
    L = [[0.0]*n for _ in range(n)]
    U = [[0.0]*n for _ in range(n)]

    for i in range(n):
        L[i][i] = 1.0   # unit diagonal

    for j in range(n):
        # Compute U[0..j][j] (upper triangular)
        for i in range(j+1):
            s = sum(L[i][k] * U[k][j] for k in range(i))
            U[i][j] = A[i][j] - s

        # Compute L[j+1..n][j] (lower triangular)
        for i in range(j+1, n):
            if abs(U[j][j]) < 1e-12:
                raise ValueError("Zero pivot encountered (matrix may require pivoting).")
            s = sum(L[i][k] * U[k][j] for k in range(j))
            L[i][j] = (A[i][j] - s) / U[j][j]

    return L, U

A = [
    [2, 1, 1],
    [4, -6, 0],
    [-2, 7, 2]
]

L, U = lu_decompose(A)

print("L =")
for r in L: print(r)

print("\nU =")
for r in U: print(r)

L =
[1.0, 0.0, 0.0]
[2.0, 1.0, 0.0]
[-1.0, -1.0, 1.0]

U =
[2, 1, 1]
[0.0, -8.0, -2.0]
[0.0, 0.0, 1.0]


In [4]:
def get_elimination_matrices(A):
    """
    Return the list of elementary elimination matrices E_k
    used to transform A into U via Gaussian elimination (no pivoting).
    A is a list of lists, square.
    Each E_k zeroes one entry below a pivot.
    """
    n = len(A)
    if any(len(row) != n for row in A):
        raise ValueError("Matrix must be square.")

    # Make a deep copy so we don't mutate A
    M = [row[:] for row in A]
    Es = []

    # Iterate over pivot columns
    for j in range(n-1):
        pivot = M[j][j]
        if abs(pivot) < 1e-12:
            raise ValueError("Zero pivot encountered (requires pivoting).")

        # Eliminate rows below pivot
        for i in range(j+1, n):
            multiplier = M[i][j] / pivot

            # Build elementary matrix E for eliminating M[i][j]
            E = [[1 if r == c else 0 for c in range(n)] for r in range(n)]
            E[i][j] = -multiplier

            # Apply elimination to M
            for c in range(n):
                M[i][c] = M[i][c] - multiplier * M[j][c]

            Es.append(E)

    return Es

A = [
    [2, 1, 1],
    [4, -6, 0],
    [-2, 7, 2]
]

Es = get_elimination_matrices(A)

for idx, E in enumerate(Es, 1):
    print(f"E{idx} =")
    for row in E:
        print(row)
    print()

E1 =
[1, 0, 0]
[-2.0, 1, 0]
[0, 0, 1]

E2 =
[1, 0, 0]
[0, 1, 0]
[1.0, 0, 1]

E3 =
[1, 0, 0]
[0, 1, 0]
[0, 1.0, 1]



In [6]:
def verify_elimination(Es, A, U, tol=1e-9):
    """
    Verify that Ek ⋯ E2 E1 A = U.
    Es: list of elimination matrices [E1, E2, ..., Ek]
    A : original matrix
    U : computed upper-triangular matrix
    Returns True/False
    """

    # Make a working copy of A
    M = [row[:] for row in A]

    # Apply each E in order: M = E @ M
    for E in Es:
        M = multiply_matrices(E, M)

    # Compare M to U
    n = len(U)
    for i in range(n):
        for j in range(n):
            if abs(M[i][j] - U[i][j]) > tol:
                return False

    return True


def multiply_matrices(A, B):
    """Basic dense matrix multiply."""
    rows = len(A)
    cols = len(B[0])
    inner = len(B)

    out = [[0]*cols for _ in range(rows)]
    for i in range(rows):
        for j in range(cols):
            out[i][j] = sum(A[i][k] * B[k][j] for k in range(inner))
    return out

Es = get_elimination_matrices(A)
L, U = lu_decompose(A)

print(verify_elimination(Es, A, U))

True



### **Explanation and Examples**

- Elimination without row exchanges gives $U$ as **upper triangular** with pivots on the diagonal.  
- $L$ is **lower triangular** with 1’s on the diagonal and multipliers $l_{ij}$ below the diagonal.  
- **Predicting zeros:**
  - If a row of $A$ starts with zeros, so does that row of $L$.  
  - If a column of $A$ starts with zeros, so does that column of $U$.  

- **Why $A = LU$?**  
  Elimination changes entries below pivots, so $A \neq U$. $L$ stores the elimination multipliers (inverse of elimination steps), and multiplying $LU$ reconstructs $A$.  

- Better balance with $LDU$:  
  $A = LU$ is "unsymmetric" because $U$ has the pivots on the diagonal and $L$ has 1’s.  
  The triangular factorization can also be written as $A = LDU$.


### **One Square System = Two Triangular Systems**

- **Why keep $L$?**  
  The matrix $L$ stores the elimination multipliers from Gaussian elimination. When a right-hand side $b$ is given, $L$ is needed to solve $Ax=b$.

- **Solve step for $Ax=b$:**
1. **Factor:** Perform elimination on $A$ to get $L$ and $U$.  
2. **Solve:**  
   - Forward elimination: $Lc = b$ (use multipliers in $L$ to update $b$ to $c$).  
   - Back substitution: $Ux = c$ to find $x$.  

- **Check solution:** Multiply $Ux = c$ by $L$:

$$
L U x = L c \quad \Rightarrow \quad Ax = b
$$


**Key Ideas**

1. Gaussian elimination (no row exchanges) factors $A$ into $L$ and $U$.  
2. $L$ contains the multipliers $l_{ij}$; multiplying $LU$ recovers $A$.  
3. Right-hand side $b$ is solved via $Lc=b$ (forward) and $Ux=c$ (backward).  
4. **Factor step:** $1/3 (n^3 - n)$ multiplications and subtractions (left side).  
5. **Solve step:** $n^2$ multiplications and subtractions (right side).
